# Import Comet for hyperparameter optimization

In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment
experiment = Experiment(
            project_name="music-genre-multiclass-classification",
            workspace="wodenwang820118",
        )
import comet_ml
import logging

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/4432c36066fe47829138d6ba6a69a943



In [2]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

# Import libraries and data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import Input,Embedding,Flatten,Concatenate,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model
%matplotlib inline
sns.set_style('whitegrid')

In [2]:
df_train_features = pd.read_csv('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

df_valid_features = pd.read_csv('valid_features.csv')
df_valid_labels = pd.read_csv('valid_labels.csv')

df_test_features = pd.read_csv('test_features.csv')

# Preprocess data

In [3]:
num_train_data = df_train_features.iloc[:,9:]
num_valid_data = df_valid_features.iloc[:,9:]
num_test_data = df_test_features.iloc[:,9:]

In [5]:
num_train_data['loudness'] = df_train_features['loudness']
num_train_data['tempo'] = df_train_features['tempo']
num_train_data['duration'] = df_train_features['duration']

num_valid_data['loudness'] = df_valid_features['loudness']
num_valid_data['tempo'] = df_valid_features['tempo']
num_valid_data['duration'] = df_valid_features['duration']

num_test_data['loudness'] = df_test_features['loudness']
num_test_data['tempo'] = df_test_features['tempo']
num_test_data['duration'] = df_test_features['duration']

In [6]:
cat_train_data = df_train_features[['time_signature', 'key', 'mode']]
cat_valid_data = df_valid_features[['time_signature', 'key', 'mode']]
cat_test_data = df_test_features[['time_signature', 'key', 'mode']]

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [8]:
cat_train_data['time_signature_cat'] = le.fit_transform(cat_train_data['time_signature'])
cat_train_data['key_cat'] = le.fit_transform(cat_train_data['key'])
cat_train_data['mode_cat'] = le.fit_transform(cat_train_data['mode'])

cat_valid_data['time_signature_cat'] = le.fit_transform(cat_valid_data['time_signature'])
cat_valid_data['key_cat'] = le.fit_transform(cat_valid_data['key'])
cat_valid_data['mode_cat'] = le.fit_transform(cat_valid_data['mode'])

cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
cat_test_data['mode_cat'] = le.fit_transform(cat_test_data['mode'])

C:\Users\alex0\AppData\Local\Temp/ipykernel_1060/3557455032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_train_data['time_signature_cat'] = le.fit_transform(cat_train_data['time_signature'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_1060/3557455032.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_train_data['key_cat'] = le.fit_transform(cat_train_data['key'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_1060/3557455032.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [9]:
cat_train_data = cat_train_data.drop(['time_signature', 'key', 'mode'], axis=1)
cat_valid_data = cat_valid_data.drop(['time_signature', 'key', 'mode'], axis=1)
cat_test_data = cat_test_data.drop(['time_signature', 'key', 'mode'], axis=1)

In [10]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [11]:
num_train = scaler.fit_transform(num_train_data)
num_valid = scaler.fit_transform(num_valid_data)
num_test = scaler.fit_transform(num_test_data)

In [12]:
num_train_y = pd.get_dummies(df_train_labels['genre'])
num_valid_y = pd.get_dummies(df_valid_labels['genre'])

# Categorical inputs

In [13]:
# size is the number of unique values in each characteristic
# size can be other values as well since it will generate numbers of vectors
time_signature_size = 6
key_size = 12
mode_size = 2

In [14]:
time_signature_input = Input(shape=(1,), name='time_signature')
key_input = Input(shape=(1,), name='key')
mode_input = Input(shape=(1,), name='mode')

In [15]:
# the min(600, round(1.6 * key_size**0.56)) is the rule of thumb for the input size(weight)
# from  fast.ai, Jeremy Howard
# reference from : https://bit.ly/3jsGw4u
time_signature_embedded = Embedding(cat_train_data.time_signature_cat.max()+1, min(600, round(1.6 * time_signature_size**0.56)), input_length=1, name='time_signature_embedded')(time_signature_input)
key_embedded = Embedding(cat_train_data.key_cat.max()+1, min(600, round(1.6 * key_size**0.56)),input_length=1,name='key_embedded')(key_input)
mode_embedded = Embedding(cat_train_data.mode_cat.max()+1, min(600, round(1.6 * mode_size**0.56)),input_length=1,name='mode_embedded')(mode_input)

In [16]:
cat_train = [cat_train_data.time_signature_cat,cat_train_data.key_cat,cat_train_data.mode_cat]
cat_valid = [cat_valid_data.time_signature_cat,cat_valid_data.key_cat,cat_valid_data.mode_cat]

# Weight adjustment and early stopping settings

In [17]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df_train_labels['genre']),y=df_train_labels['genre'])
class_weight = {i :class_weight[i] for i in range(8)}

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5)

# Building model

In [22]:
class NumericGenreClassifier:
    def __init__(self,num_train,cat_train,num_train_y,num_valid,cat_valid,num_valid_y,class_weight,early_stop,experiment):
        # numeric and label input
        self.num_train = num_train
        self.num_train_y = num_train_y
        self.num_valid = num_valid
        self.num_valid_y = num_valid_y

        # categorical input
        self.cat_train = cat_train
        self.cat_valid = cat_valid

        # class weight and early stop mechanism
        self.class_weight = class_weight
        self.early_stop = early_stop

        # comet_ml experiment
        self.experiment = experiment
    
    def build_model(self):
        # numeric input layers
        numerical_input = Input(shape=(self.num_train.shape[1],), name='numerical_input')
        x = Dense(
            self.experiment.get_parameter("num_first_layer_units"),
            activation='elu',
            kernel_initializer='HeNormal')(numerical_input)
        x = Dropout(0.5)(x)
        x = BatchNormalization()(x)

        x = Dense(
            self.experiment.get_parameter("num_second_layer_units"),
            activation='elu',
            kernel_initializer='HeNormal')(x)
        x = Dropout(0.3)(x)
        x = BatchNormalization()(x)
        numerical_output = Flatten()(x) 

        # categorical input layers
        categorical_input = [time_signature_input,key_input,mode_input]
        concatenated = Concatenate(name='category')([time_signature_embedded, key_embedded, mode_embedded])
        embedded_input = Flatten()(concatenated)
        y = Dense(
            units=self.experiment.get_parameter("cat_first_layer_units"),
            activation='elu',
            kernel_initializer='HeUniform')(embedded_input)
        y = Dropout(0.5)(y)
        y = BatchNormalization()(y)

        y = Dense(
            units=self.experiment.get_parameter("cat_second_layer_units"),
            activation='elu',
            kernel_initializer='HeUniform')(y)
        y = Dropout(0.3)(y)
        y = BatchNormalization()(y)
        cat_out = y

        # final layer and output
        final_layer = Concatenate()([numerical_output,cat_out])
        final_layer = Dense(
                        units=self.experiment.get_parameter("final_layer_units"),
                        activation='elu',
                        kernel_initializer='HeNormal')(final_layer)

        final_model_output = Dense(8,activation='softmax', kernel_initializer='HeNormal')(final_layer)

        model = Model(inputs=[numerical_input, categorical_input],outputs=final_model_output,name='final_output')
        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    def train_model(self):
        # Train the model
        model = self.build_model()
        model.fit(
            [self.num_train, self.cat_train],
            self.num_train_y,
            batch_size=self.experiment.get_parameter("batch_size"),
            epochs=self.experiment.get_parameter("epochs"),
            validation_data=([self.num_valid,self.cat_valid],self.num_valid_y),
            class_weight=self.class_weight,
            callbacks=[self.early_stop]
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        # score = model.evaluate(self.num_valid,self.num_valid_y)
        # LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="music-genre-multiclass-classification"):

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [23]:
numerical_model = NumericGenreClassifier(num_train,cat_train,num_train_y,num_valid,cat_valid,num_valid_y,class_weight,early_stop,experiment)
# mu is the mean number of units, sigma is the standard deviation
numerical_model.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "num_first_layer_units": {"type": "discrete", "values": [400,450,500,550,600,700,800,900,1000]},
        "num_second_layer_units": {"type": "discrete", "values": [100,150,200,250,300,350]},
        "cat_first_layer_units": {"type": "discrete", "values": [100,150,200,250,300,350]},
        "cat_second_layer_units": {"type": "discrete", "values": [30,50,100,150,200]},
        "final_layer_units": {"type": "discrete", "values": [30,40,60,100,150,200,250,300,350]},
        "batch_size": {"type": "discrete", "values": [16, 32, 64]},
        "epochs": {"type": "discrete", "values": [9,10,15,20]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=1b2c56a2473641f1a1193482bd9c6cdb
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 174960, 'endTime': None, 'id': '1b2c56a2473641f1a1193482bd9c6cdb', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [16, 32, 64]}, 'cat_first_layer_units': {'type': 'discrete', 'values': [100, 150, 200, 250, 300, 350]}, 'cat_second_layer_units': {'type': 'discrete', 'values': [30, 50, 100, 150, 200]}, 'epochs': {'type': 'discrete', 'values': [9, 10, 15, 20]}, 'final_layer_units': {'type': 'discrete', 'values': [30, 40, 60, 100, 150, 200, 250, 300, 350]}, 'num_first_layer_units': {'type': 'discrete', 'values': [400, 450, 500, 550, 600, 700, 800, 900, 1000]}, 'num_second_layer_units': {'type': 'discrete', 'values': [100, 150, 200, 250, 300, 350]}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 10, 'metric': 'loss', 'minSampleSize': 100, 'ob

Epoch 1/10
120/120 [==============================] - 2s 10ms/step - loss: 1.7164 - accuracy: 0.4031 - val_loss: 1.4254 - val_accuracy: 0.4978
Epoch 2/10
120/120 [==============================] - 1s 7ms/step - loss: 1.3786 - accuracy: 0.4870 - val_loss: 1.4172 - val_accuracy: 0.4756
Epoch 3/10
120/120 [==============================] - 1s 7ms/step - loss: 1.2639 - accuracy: 0.5253 - val_loss: 1.3782 - val_accuracy: 0.4867
Epoch 4/10
120/120 [==============================] - 1s 7ms/step - loss: 1.1943 - accuracy: 0.5428 - val_loss: 1.3151 - val_accuracy: 0.5022
Epoch 5/10
120/120 [==============================] - 1s 7ms/step - loss: 1.1494 - accuracy: 0.5597 - val_loss: 1.3006 - val_accuracy: 0.5111
Epoch 6/10
120/120 [==============================] - 1s 7ms/step - loss: 1.1147 - accuracy: 0.5762 - val_loss: 1.3282 - val_accuracy: 0.5311
Epoch 7/10
120/120 [==============================] - 1s 7ms/step - loss: 1.0712 - accuracy: 0.5961 - val_loss: 1.2976 - val_accuracy: 0.5289
Epoch

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/2f37a1f5eaf3495a9e54da779e9d79b7
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [18]                : (0.40049493312835693, 0.6178692579269409)
COMET INFO:     batch_accuracy [216]         : (0.046875, 0.671875)
COMET INFO:     batch_loss [216]             : (0.8400017023086548, 3.5377562046051025)
COMET INFO:     epoch_duration [18]          : (0.8589999999967404, 2.0310000000026776)
COMET INFO:     loss [18]                    : (0.9859532713890076, 1.7164337635040283)
COMET INFO:     val_accuracy [18]            : (0.4555555582046509, 0.5577777624130249)
COMET INFO:     val_loss [18]                : (1.2976216077804565, 1.4712800979614258)
COMET INFO:     validate_batch_

Epoch 1/9
480/480 [==============================] - 8s 14ms/step - loss: 1.8882 - accuracy: 0.3821 - val_loss: 1.5143 - val_accuracy: 0.4778
Epoch 2/9
480/480 [==============================] - 6s 13ms/step - loss: 1.5364 - accuracy: 0.4575 - val_loss: 1.5168 - val_accuracy: 0.4356
Epoch 3/9
480/480 [==============================] - 6s 13ms/step - loss: 1.4023 - accuracy: 0.4822 - val_loss: 1.4049 - val_accuracy: 0.5067
Epoch 4/9
480/480 [==============================] - 6s 13ms/step - loss: 1.3343 - accuracy: 0.5004 - val_loss: 1.3646 - val_accuracy: 0.5333
Epoch 5/9
480/480 [==============================] - 7s 14ms/step - loss: 1.2671 - accuracy: 0.5221 - val_loss: 1.4230 - val_accuracy: 0.5000
Epoch 6/9
480/480 [==============================] - 6s 13ms/step - loss: 1.2488 - accuracy: 0.5300 - val_loss: 1.4502 - val_accuracy: 0.5000
Epoch 7/9
480/480 [==============================] - 6s 13ms/step - loss: 1.2114 - accuracy: 0.5406 - val_loss: 1.4888 - val_accuracy: 0.4956
Epoch 

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/71f015d371fc40e59db5ad6bcf298ba6
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [18]                : (0.377702534198761, 0.5636884570121765)
COMET INFO:     batch_accuracy [864]         : (0.0625, 0.8125)
COMET INFO:     batch_loss [864]             : (0.46002256870269775, 3.063964605331421)
COMET INFO:     epoch_duration [18]          : (6.10899999999674, 8.203000000001339)
COMET INFO:     loss [18]                    : (1.152809500694275, 1.8938027620315552)
COMET INFO:     val_accuracy [18]            : (0.41777777671813965, 0.5377777814865112)
COMET INFO:     val_loss [18]                : (1.3026564121246338, 1.6303632259368896)
COMET INFO:     validate_batch_accuracy 

...

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/fa3b3f9354f847eb807e34331f5f1294
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [17]                : (0.3924199044704437, 0.5867413282394409)
COMET INFO:     batch_accuracy [408]         : (0.15625, 0.6875)
COMET INFO:     batch_loss [408]             : (0.7310987710952759, 2.981051445007324)
COMET INFO:     epoch_duration [17]          : (3.125, 4.89100000000326)
COMET INFO:     loss [17]                    : (1.06450617313385, 1.8180673122406006)
COMET INFO:     val_accuracy [17]            : (0.46222221851348877, 0.5688889026641846)
COMET INFO:     val_loss [17]                : (1.3021456003189087, 1.6097370386123657)
COMET INFO:     validate_batch_accuracy [34] : (0.31

Epoch 1/15
240/240 [==============================] - 5s 15ms/step - loss: 1.8082 - accuracy: 0.3893 - val_loss: 1.5278 - val_accuracy: 0.4933
Epoch 2/15
240/240 [==============================] - 3s 15ms/step - loss: 1.4325 - accuracy: 0.4780 - val_loss: 1.3354 - val_accuracy: 0.5400
Epoch 3/15
240/240 [==============================] - 3s 14ms/step - loss: 1.3177 - accuracy: 0.5096 - val_loss: 1.3475 - val_accuracy: 0.5089
Epoch 4/15
240/240 [==============================] - 3s 13ms/step - loss: 1.2607 - accuracy: 0.5289 - val_loss: 1.4332 - val_accuracy: 0.4978
Epoch 5/15
240/240 [==============================] - 3s 13ms/step - loss: 1.1948 - accuracy: 0.5440 - val_loss: 1.3032 - val_accuracy: 0.5600
Epoch 6/15
240/240 [==============================] - 3s 13ms/step - loss: 1.1424 - accuracy: 0.5646 - val_loss: 1.4634 - val_accuracy: 0.4844
Epoch 7/15
240/240 [==============================] - 3s 13ms/step - loss: 1.1144 - accuracy: 0.5762 - val_loss: 1.3444 - val_accuracy: 0.5378

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/8ed1e35bc6c8454bbe38a7f34743c794
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [18]                : (0.3892940878868103, 0.6100547313690186)
COMET INFO:     batch_accuracy [432]         : (0.0625, 0.75)
COMET INFO:     batch_loss [432]             : (0.7861683368682861, 4.462146759033203)
COMET INFO:     epoch_duration [18]          : (3.063000000009197, 5.4689999999973224)
COMET INFO:     loss [18]                    : (1.004556655883789, 1.8081704378128052)
COMET INFO:     val_accuracy [18]            : (0.42444443702697754, 0.5600000023841858)
COMET INFO:     val_loss [18]                : (1.3032193183898926, 1.6435524225234985)
COMET INFO:     validate_batch_accuracy 

Epoch 1/15
480/480 [==============================] - 8s 14ms/step - loss: 1.8578 - accuracy: 0.3647 - val_loss: 1.5378 - val_accuracy: 0.4400
Epoch 2/15
480/480 [==============================] - 6s 13ms/step - loss: 1.5187 - accuracy: 0.4478 - val_loss: 1.4343 - val_accuracy: 0.5111
Epoch 3/15
480/480 [==============================] - 7s 14ms/step - loss: 1.4105 - accuracy: 0.4760 - val_loss: 1.3792 - val_accuracy: 0.5267
Epoch 4/15
480/480 [==============================] - 7s 14ms/step - loss: 1.3460 - accuracy: 0.5030 - val_loss: 1.4286 - val_accuracy: 0.4756
Epoch 5/15
480/480 [==============================] - 6s 13ms/step - loss: 1.2975 - accuracy: 0.5188 - val_loss: 1.4710 - val_accuracy: 0.4822
Epoch 6/15
480/480 [==============================] - 6s 13ms/step - loss: 1.2637 - accuracy: 0.5292 - val_loss: 1.4060 - val_accuracy: 0.4733
Epoch 7/15
480/480 [==============================] - 7s 14ms/step - loss: 1.2319 - accuracy: 0.5302 - val_loss: 1.3788 - val_accuracy: 0.5156

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/music-genre-multiclass-classification/7394c86fba8d46c6ab04c26e90771b0e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [29]                : (0.364678293466568, 0.5781453251838684)
COMET INFO:     batch_accuracy [1392]        : (0.0625, 0.75)
COMET INFO:     batch_loss [1392]            : (0.5708247423171997, 3.287818431854248)
COMET INFO:     epoch_duration [29]          : (6.14100000000326, 8.452999999994063)
COMET INFO:     loss [29]                    : (1.0818830728530884, 1.8578357696533203)
COMET INFO:     val_accuracy [29]            : (0.402222216129303, 0.5400000214576721)
COMET INFO:     val_loss [29]                : (1.291308879852295, 1.661255955696106)
COMET INFO:     validate_batch_accuracy [87] :

In [19]:
# numeric input layers
numerical_input = Input(shape=(num_train.shape[1],), name='numerical_input')
x = Dense(
    1000,
    activation='elu',
    kernel_initializer='HeNormal')(numerical_input)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

x = Dense(
    200,
    activation='elu',
    kernel_initializer='HeNormal')(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
numerical_output = Flatten()(x) 

# categorical input layers
categorical_input = [time_signature_input,key_input,mode_input]
concatenated = Concatenate(name='category')([time_signature_embedded, key_embedded, mode_embedded])
embedded_input = Flatten()(concatenated)
y = Dense(
    units=250,
    activation='elu',
    kernel_initializer='HeUniform')(embedded_input)
y = Dropout(0.5)(y)
y = BatchNormalization()(y)

y = Dense(
    units=50,
    activation='elu',
    kernel_initializer='HeUniform')(y)
y = Dropout(0.3)(y)
y = BatchNormalization()(y)
cat_out = y

# final layer and output
final_layer = Concatenate()([numerical_output,cat_out])
final_layer = Dense(
                units=40,
                activation='elu',
                kernel_initializer='HeNormal')(final_layer)

final_model_output = Dense(8,activation='softmax', kernel_initializer='HeNormal')(final_layer)

model = Model(inputs=[numerical_input, categorical_input],outputs=final_model_output,name='final_output')
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [20]:
model.fit(
    [num_train,cat_train],
    num_train_y,
    batch_size=64,
    epochs=9,
    validation_data=([num_valid,cat_valid],num_valid_y),
    class_weight=class_weight,
    callbacks=[early_stop]
)

Epoch 1/9
120/120 [==============================] - 3s 10ms/step - loss: 1.6760 - accuracy: 0.4032 - val_loss: 1.6187 - val_accuracy: 0.4444
Epoch 2/9
120/120 [==============================] - 1s 7ms/step - loss: 1.3464 - accuracy: 0.5057 - val_loss: 1.3828 - val_accuracy: 0.5244
Epoch 3/9
120/120 [==============================] - 1s 7ms/step - loss: 1.2469 - accuracy: 0.5371 - val_loss: 1.3917 - val_accuracy: 0.5067
Epoch 4/9
120/120 [==============================] - 1s 7ms/step - loss: 1.1680 - accuracy: 0.5567 - val_loss: 1.3767 - val_accuracy: 0.5089
Epoch 5/9
120/120 [==============================] - 1s 7ms/step - loss: 1.1284 - accuracy: 0.5766 - val_loss: 1.2903 - val_accuracy: 0.5467
Epoch 6/9
120/120 [==============================] - 1s 7ms/step - loss: 1.0904 - accuracy: 0.5763 - val_loss: 1.2903 - val_accuracy: 0.5333
Epoch 7/9
120/120 [==============================] - 1s 7ms/step - loss: 1.0242 - accuracy: 0.6068 - val_loss: 1.4228 - val_accuracy: 0.4822
Epoch 8/9
12

In [21]:
prediction = model.predict([num_valid,cat_valid])

In [22]:
predict_labels = [ label.argmax() for label in prediction]
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
predict_label_array = np.vectorize(classes_mapping.get)(predict_labels)
correct_label = df_valid_labels['genre'].values

In [23]:
from sklearn.metrics import classification_report
print(classification_report(correct_label, predict_label_array))

                       precision    recall  f1-score   support

 classic pop and rock       0.45      0.44      0.44        55
dance and electronica       0.38      0.44      0.41        45
                 folk       0.38      0.61      0.47        64
       jazz and blues       0.47      0.39      0.42        44
                metal       0.82      0.70      0.75        66
                  pop       0.70      0.50      0.58        74
                 punk       0.51      0.57      0.54        44
      soul and reggae       0.66      0.53      0.59        58

             accuracy                           0.53       450
            macro avg       0.55      0.52      0.53       450
         weighted avg       0.56      0.53      0.54       450



In [24]:
# model.save('audio_meta_model')